# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 8 2022 10:30AM,239110,16,8117026,Sartorius Lab Products and Service,Released
1,Jun 8 2022 10:22AM,239092,16,8110511,Sartorius Bioprocess Solutions,Released
2,Jun 8 2022 10:22AM,239092,16,8112588,Sartorius Bioprocess Solutions,Released
3,Jun 8 2022 10:21AM,239107,18,6558574,Mimi Luzon LTD,Released
4,Jun 8 2022 10:20AM,239108,10,2-100309,Emerginnova,Released
5,Jun 8 2022 10:20AM,239106,21,399290087,"NBTY Global, Inc.",Released
6,Jun 8 2022 10:08AM,239095,21,399290053,"NBTY Global, Inc.",Released
7,Jun 8 2022 10:02AM,239094,19,ACG-2102490278,ACG North America LLC,Released
8,Jun 8 2022 10:02AM,239094,19,ACG-2102490277,ACG North America LLC,Released
9,Jun 8 2022 10:01AM,239093,15,7913461-BO,"Mizner Bioscience, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
52,239095,Released,1
53,239106,Released,1
54,239107,Released,1
55,239108,Released,1
56,239110,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239095,NaN,NaN,1.0
239106,NaN,NaN,1.0
239107,NaN,NaN,1.0
239108,NaN,NaN,1.0
239110,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238984,0.0,2.0,0.0
238989,0.0,1.0,0.0
238992,0.0,0.0,1.0
238994,8.0,50.0,5.0
238998,0.0,0.0,16.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238984,0,2,0
238989,0,1,0
238992,0,0,1
238994,8,50,5
238998,0,0,16


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238984,0,2,0
1,238989,0,1,0
2,238992,0,0,1
3,238994,8,50,5
4,238998,0,0,16


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238984,,2,
1,238989,,1,
2,238992,,,1
3,238994,8,50,5
4,238998,,,16


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 8 2022 10:30AM,239110,16,Sartorius Lab Products and Service
1,Jun 8 2022 10:22AM,239092,16,Sartorius Bioprocess Solutions
3,Jun 8 2022 10:21AM,239107,18,Mimi Luzon LTD
4,Jun 8 2022 10:20AM,239108,10,Emerginnova
5,Jun 8 2022 10:20AM,239106,21,"NBTY Global, Inc."
6,Jun 8 2022 10:08AM,239095,21,"NBTY Global, Inc."
7,Jun 8 2022 10:02AM,239094,19,ACG North America LLC
9,Jun 8 2022 10:01AM,239093,15,"Mizner Bioscience, LLC"
23,Jun 8 2022 9:56AM,239050,15,"Person & Covey, Inc."
35,Jun 8 2022 9:55AM,239091,18,Eye Pharma Inc


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 8 2022 10:30AM,239110,16,Sartorius Lab Products and Service,,,1
1,Jun 8 2022 10:22AM,239092,16,Sartorius Bioprocess Solutions,,,2
2,Jun 8 2022 10:21AM,239107,18,Mimi Luzon LTD,,,1
3,Jun 8 2022 10:20AM,239108,10,Emerginnova,,,1
4,Jun 8 2022 10:20AM,239106,21,"NBTY Global, Inc.",,,1
5,Jun 8 2022 10:08AM,239095,21,"NBTY Global, Inc.",,,1
6,Jun 8 2022 10:02AM,239094,19,ACG North America LLC,,,2
7,Jun 8 2022 10:01AM,239093,15,"Mizner Bioscience, LLC",,,14
8,Jun 8 2022 9:56AM,239050,15,"Person & Covey, Inc.",,,12
9,Jun 8 2022 9:55AM,239091,18,Eye Pharma Inc,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 8 2022 10:30AM,239110,16,Sartorius Lab Products and Service,1,,
1,Jun 8 2022 10:22AM,239092,16,Sartorius Bioprocess Solutions,2,,
2,Jun 8 2022 10:21AM,239107,18,Mimi Luzon LTD,1,,
3,Jun 8 2022 10:20AM,239108,10,Emerginnova,1,,
4,Jun 8 2022 10:20AM,239106,21,"NBTY Global, Inc.",1,,
5,Jun 8 2022 10:08AM,239095,21,"NBTY Global, Inc.",1,,
6,Jun 8 2022 10:02AM,239094,19,ACG North America LLC,2,,
7,Jun 8 2022 10:01AM,239093,15,"Mizner Bioscience, LLC",14,,
8,Jun 8 2022 9:56AM,239050,15,"Person & Covey, Inc.",12,,
9,Jun 8 2022 9:55AM,239091,18,Eye Pharma Inc,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 8 2022 10:30AM,239110,16,Sartorius Lab Products and Service,1,,
1,Jun 8 2022 10:22AM,239092,16,Sartorius Bioprocess Solutions,2,,
2,Jun 8 2022 10:21AM,239107,18,Mimi Luzon LTD,1,,
3,Jun 8 2022 10:20AM,239108,10,Emerginnova,1,,
4,Jun 8 2022 10:20AM,239106,21,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 8 2022 10:30AM,239110,16,Sartorius Lab Products and Service,1.0,NaN,NaN
1,Jun 8 2022 10:22AM,239092,16,Sartorius Bioprocess Solutions,2.0,NaN,NaN
2,Jun 8 2022 10:21AM,239107,18,Mimi Luzon LTD,1.0,NaN,NaN
3,Jun 8 2022 10:20AM,239108,10,Emerginnova,1.0,NaN,NaN
4,Jun 8 2022 10:20AM,239106,21,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 8 2022 10:30AM,239110,16,Sartorius Lab Products and Service,1.0,0.0,0.0
1,Jun 8 2022 10:22AM,239092,16,Sartorius Bioprocess Solutions,2.0,0.0,0.0
2,Jun 8 2022 10:21AM,239107,18,Mimi Luzon LTD,1.0,0.0,0.0
3,Jun 8 2022 10:20AM,239108,10,Emerginnova,1.0,0.0,0.0
4,Jun 8 2022 10:20AM,239106,21,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2151601,161.0,12.0,26.0
102,717254,3.0,0.0,0.0
12,717154,2.0,2.0,0.0
15,1434328,84.0,4.0,0.0
16,478202,3.0,0.0,0.0
18,717214,3.0,0.0,0.0
19,1912364,41.0,9.0,0.0
20,1195093,51.0,52.0,8.0
21,2151445,9.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2151601,161.0,12.0,26.0
1,102,717254,3.0,0.0,0.0
2,12,717154,2.0,2.0,0.0
3,15,1434328,84.0,4.0,0.0
4,16,478202,3.0,0.0,0.0
5,18,717214,3.0,0.0,0.0
6,19,1912364,41.0,9.0,0.0
7,20,1195093,51.0,52.0,8.0
8,21,2151445,9.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,161.0,12.0,26.0
1,102,3.0,0.0,0.0
2,12,2.0,2.0,0.0
3,15,84.0,4.0,0.0
4,16,3.0,0.0,0.0
5,18,3.0,0.0,0.0
6,19,41.0,9.0,0.0
7,20,51.0,52.0,8.0
8,21,9.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,161.0
1,102,Released,3.0
2,12,Released,2.0
3,15,Released,84.0
4,16,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,18,19,20,21
Status,,,,,,,,,
Completed,26.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0
Executing,12.0,0.0,2.0,4.0,0.0,0.0,9.0,52.0,0.0
Released,161.0,3.0,2.0,84.0,3.0,3.0,41.0,51.0,9.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,18,19,20,21
0,Completed,26.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0
1,Executing,12.0,0.0,2.0,4.0,0.0,0.0,9.0,52.0,0.0
2,Released,161.0,3.0,2.0,84.0,3.0,3.0,41.0,51.0,9.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,18,19,20,21
0,Completed,26.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0
1,Executing,12.0,0.0,2.0,4.0,0.0,0.0,9.0,52.0,0.0
2,Released,161.0,3.0,2.0,84.0,3.0,3.0,41.0,51.0,9.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()